### Transfer Learning

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 사전 학습된 모델 로드 (Imagenet 가중치 사용)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 새로운 모델 생성
model = Model(inputs=base_model.input, outputs=predictions)

# 모든 레이어 학습 가능하도록 설정
for layer in base_model.layers:
    layer.trainable = True

#FC Layer와 Convolution Layer의 다른 learning rate 설정
optimizer = Adam(learning_rate=1e-4)
optimizer_fine_tune = Adam(learning_rate=1e-5)

#모델 컴파일
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터셋 준비 (CIFAR-10 예시)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = tf.image.resize(x_train, (224, 224)) / 255.0
x_test = tf.image.resize(x_test, (224, 224)) / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 모델 학습
history = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=32
)

### Knowledge Distillation

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

# Teacher 모델 정의 (복잡한 모델)
teacher_model = Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Teacher 모델 컴파일 및 학습
teacher_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# MNIST 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = np.expand_dims(x_train, -1)  # 채널 추가
x_test = np.expand_dims(x_test, -1)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Teacher 모델 학습
teacher_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

# Teacher 모델의 soft label 생성
temperature = 5.0  # Softmax temperature
teacher_logits = teacher_model.predict(x_train)
soft_labels = tf.nn.softmax(teacher_logits / temperature).numpy()

# Student 모델 정의 (간단한 모델)
student_model = Sequential([
    Flatten(input_shape=(28, 28, 1)),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Knowledge Distillation loss 함수 정의
def distillation_loss(y_true, y_pred, teacher_output, temperature):
    y_true_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    soft_loss = tf.keras.losses.categorical_crossentropy(
        tf.nn.softmax(teacher_output / temperature),
        tf.nn.softmax(y_pred / temperature)
    )
    return y_true_loss + temperature**2 * soft_loss

# Student 모델 학습
optimizer = Adam(learning_rate=0.001)
for epoch in range(5):
    for i in range(0, len(x_train), 64):
        x_batch = x_train[i:i+64]
        y_batch = y_train[i:i+64]
        soft_batch = soft_labels[i:i+64]
        with tf.GradientTape() as tape:
            student_logits = student_model(x_batch, training=True)
            loss = distillation_loss(y_batch, student_logits, soft_batch, temperature)
        grads = tape.gradient(loss, student_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, student_model.trainable_variables))

# Student 모델 평가
student_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
student_model.evaluate(x_test, y_test)


Epoch 1/5
938/938 [==============================] - 35s 36ms/step - loss: 0.1311 - accuracy: 0.9603 - val_loss: 0.0481 - val_accuracy: 0.9848
Epoch 2/5
938/938 [==============================] - 42s 45ms/step - loss: 0.0407 - accuracy: 0.9873 - val_loss: 0.0372 - val_accuracy: 0.9875
Epoch 3/5
938/938 [==============================] - 38s 41ms/step - loss: 0.0280 - accuracy: 0.9911 - val_loss: 0.0287 - val_accuracy: 0.9895
Epoch 4/5
938/938 [==============================] - 38s 40ms/step - loss: 0.0194 - accuracy: 0.9937 - val_loss: 0.0289 - val_accuracy: 0.9906
Epoch 5/5
313/313 [==============================] - 1s 1ms/step - loss: 0.1170 - accuracy: 0.9641


[0.11702550202608109, 0.9641000032424927]

### GoogLeNet

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense, Input, Dropout, concatenate
from tensorflow.keras.models import Model

# Inception Module 정의
def inception_module(x, f1, f3_in, f3, f5_in, f5, pool_proj):
    # 1x1 Conv
    conv1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    # 1x1 Conv -> 3x3 Conv
    conv3 = Conv2D(f3_in, (1, 1), padding='same', activation='relu')(x)
    conv3 = Conv2D(f3, (3, 3), padding='same', activation='relu')(conv3)

    # 1x1 Conv -> 5x5 Conv
    conv5 = Conv2D(f5_in, (1, 1), padding='same', activation='relu')(x)
    conv5 = Conv2D(f5, (5, 5), padding='same', activation='relu')(conv5)

    # MaxPooling -> 1x1 Conv
    pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = Conv2D(pool_proj, (1, 1), padding='same', activation='relu')(pool)

    # Concat 모든 branch
    output = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return output

# Auxiliary Classifier 정의
def auxiliary_classifier(x, num_classes):
    x = AveragePooling2D((5, 5), strides=(3, 3))(x)
    x = Conv2D(128, (1, 1), padding='same', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.7)(x)
    x = Dense(num_classes, activation='softmax')(x)
    return x

# GoogLeNet 모델 정의
def googlenet(input_shape=(224, 224, 3), num_classes=1000):
    inputs = Input(shape=input_shape)

    # Stem Network
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Inception Modules
    x = inception_module(x, 64, 96, 128, 16, 32, 32)
    x = inception_module(x, 128, 128, 192, 32, 96, 64)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, 192, 96, 208, 16, 48, 64)
    aux1 = auxiliary_classifier(x, num_classes)  # Auxiliary Classifier 1

    x = inception_module(x, 160, 112, 224, 24, 64, 64)
    x = inception_module(x, 128, 128, 256, 24, 64, 64)
    x = inception_module(x, 112, 144, 288, 32, 64, 64)
    aux2 = auxiliary_classifier(x, num_classes)  # Auxiliary Classifier 2

    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, 256, 160, 320, 32, 128, 128)
    x = inception_module(x, 384, 192, 384, 48, 128, 128)

    # Final Output
    x = AveragePooling2D((7, 7), strides=(1, 1))(x)
    x = Flatten()(x)
    x = Dropout(0.4)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # 모델 정의 (Auxiliary Classifier 포함)
    model = Model(inputs, [outputs, aux1, aux2])
    return model

# GoogLeNet 모델 생성
model = googlenet(input_shape=(224, 224, 3), num_classes=10)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                              

### Dense Block

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, AveragePooling2D, GlobalAveragePooling2D, Dense, Input, concatenate
from tensorflow.keras.models import Model

# Dense Block 정의
def dense_block(x, num_layers, growth_rate):
    for _ in range(num_layers):
        # Bottleneck Layer: 1x1 Conv로 채널 수를 줄여 계산량 감소
        bn = BatchNormalization()(x)  # 입력 정규화
        relu = ReLU()(bn)  # 활성화 함수 적용
        conv = Conv2D(4 * growth_rate, (1, 1), padding='same', activation=None)(relu)  # 1x1 Convolution

        # 3x3 Convolution Layer
        bn = BatchNormalization()(conv)  # 정규화
        relu = ReLU()(bn)  # 활성화 함수 적용
        conv = Conv2D(growth_rate, (3, 3), padding='same', activation=None)(relu)  # 3x3 Convolution

        # Concatenate: 이전 레이어의 출력과 현재 레이어의 출력을 연결
        x = concatenate([x, conv], axis=-1)
    return x

# Transition Layer 정의
def transition_layer(x, reduction):
    # 채널 수 줄이기
    bn = BatchNormalization()(x)  # 정규화
    relu = ReLU()(bn)  # 활성화 함수 적용
    num_filters = int(x.shape[-1] * reduction)  # 채널 수 감소 비율 적용
    conv = Conv2D(num_filters, (1, 1), padding='same', activation=None)(relu)  # 1x1 Convolution

    # 공간 크기 줄이기
    pool = AveragePooling2D((2, 2), strides=2)(conv)  # 2x2 영역 평균 풀링
    return pool

# DenseNet 모델 정의
def densenet(input_shape=(224, 224, 3), num_classes=1000, growth_rate=32, num_blocks=4, num_layers_per_block=[6, 12, 24, 16]):
    inputs = Input(shape=input_shape)  # 입력 정의

    # 초기 Convolution Layer
    x = Conv2D(2 * growth_rate, (7, 7), strides=2, padding='same', activation=None)(inputs)  # 7x7 Conv
    x = BatchNormalization()(x)  # 정규화
    x = ReLU()(x)  # 활성화 함수
    x = AveragePooling2D((3, 3), strides=2, padding='same')(x)  # 3x3 Average Pooling

    # Dense Blocks and Transition Layers
    for i in range(num_blocks):
        x = dense_block(x, num_layers=num_layers_per_block[i], growth_rate=growth_rate)  # Dense Block 추가
        if i != num_blocks - 1:  # 마지막 블록 이후에는 Transition Layer 없음
            x = transition_layer(x, reduction=0.5)  # Transition Layer 추가

    # Global Average Pooling and Fully Connected Layer
    x = GlobalAveragePooling2D()(x)  # Feature 맵을 하나의 벡터로 변환
    outputs = Dense(num_classes, activation='softmax')(x)  # 클래스 예측

    # 모델 생성
    model = Model(inputs, outputs)
    return model

# DenseNet 모델 생성
model = densenet(input_shape=(224, 224, 3), num_classes=10, growth_rate=32)
model.summary()  # 모델 구조 출력


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_61 (Conv2D)             (None, 112, 112, 64  9472        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 112, 112, 64  256        ['conv2d_61[0][0]']              
 alization)                     )                                                           

### SENet

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, Reshape, Multiply, Input
from tensorflow.keras.models import Model

# Squeeze-and-Excitation Block 정의
def se_block(input_tensor, reduction=16):
    """
    Squeeze-and-Excitation Block
    Args:
        input_tensor: 입력 텐서
        reduction: 채널 축소 비율
    Returns:
        output_tensor: SE Block이 적용된 텐서
    """
    # 채널 수 가져오기
    channels = input_tensor.shape[-1]

    # Squeeze: Global Average Pooling
    squeeze = GlobalAveragePooling2D()(input_tensor)

    # Excitation: Fully Connected 레이어
    excitation = Dense(channels // reduction, activation='relu')(squeeze)  # 축소 단계
    excitation = Dense(channels, activation='sigmoid')(excitation)  # 복원 단계

    # Excitation 값을 채널 차원에 맞게 Reshape
    excitation = Reshape((1, 1, channels))(excitation)

    # 입력 텐서와 Excitation 값을 곱하여 조정
    output_tensor = Multiply()([input_tensor, excitation])
    return output_tensor

# 간단한 CNN 모델에 SE Block 추가
def create_se_cnn(input_shape=(224, 224, 3), num_classes=10):
    inputs = Input(shape=input_shape)

    # Convolution Layer
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)

    # SE Block 적용
    x = se_block(x)

    # 추가 Convolution Layer
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)

    # SE Block 적용
    x = se_block(x)

    # Global Average Pooling과 출력
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    # 모델 생성
    model = Model(inputs, outputs)
    return model

# 모델 생성
model = create_se_cnn(input_shape=(224, 224, 3), num_classes=10)
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_181 (Conv2D)            (None, 224, 224, 64  1792        ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 64)          0           ['conv2d_181[0][0]']             
 obalAveragePooling2D)                                                                      

### EfficientNet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model

# EfficientNet 기반 모델 정의
def create_efficientnet(input_shape=(224, 224, 3), num_classes=10):
    """
    EfficientNet 기반 모델 생성
    Args:
        input_shape: 입력 이미지 크기 (Height, Width, Channels)
        num_classes: 출력 클래스 수
    Returns:
        model: EfficientNet 기반 모델
    """

    
    # *** EfficientNetB0 사전 학습된 모델 로드 (ImageNet 가중치 사용)
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

    # Feature Extraction (기본 모델 출력 가져오기)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # Feature 맵을 벡터로 변환

    # Fully Connected Layer 추가
    outputs = Dense(num_classes, activation='softmax')(x)

    # 전체 모델 생성
    model = Model(inputs=base_model.input, outputs=outputs)

    # *** 사전 학습된 레이어 고정 (Fine-tuning 시 고정 해제 가능)
    for layer in base_model.layers:
        layer.trainable = False

    return model

# 모델 생성
model = create_efficientnet(input_shape=(224, 224, 3), num_classes=10)

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 구조 출력
model.summary()

# 데이터 준비 (CIFAR-10 예제)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = tf.image.resize(x_train / 255.0, (224, 224))  # EfficientNet 입력 크기로 리사이즈
x_test = tf.image.resize(x_test / 255.0, (224, 224))
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# 모델 학습
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=32)
